<a href="https://colab.research.google.com/github/Berkay86/DeepPerson/blob/main/CHESSGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import cv2
import zipfile
import os

# Zip dosyasını aç
with zipfile.ZipFile("Videolarım.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Klasörlerin adlarını saklamak için bir liste oluştur
folder_names = []

# Zip dosyasındaki her dosya için
for file in os.listdir():
    if file.endswith(".mp4"):
        # Video dosyasını aç
        cap = cv2.VideoCapture(file)

        # Klasör adı oluştur
        folder_name = os.path.splitext(file)[0]
        folder_names.append(folder_name)

        # Klasör oluştur
        os.makedirs(folder_name)

        # Her frame için
        i = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret == False:
                break
            
            # Sadece 5., 6., 7., ... sn'lik frame'leri kaydet
            if i % 30 == 0:
                cv2.imwrite(folder_name + "/" + folder_name + str(i) + '.jpg', frame)
            i += 1

        # Video dosyasını kapat
        cap.release()
        cv2.destroyAllWindows()



# Tüm klasörleri ve içeriğini bir zip dosyasına kaydet
with zipfile.ZipFile("result.zip", 'w') as zip_file:
    for folder_name in folder_names:
        for root, dirs, files in os.walk(folder_name):
            for file in files:
                zip_file.write(os.path.join(root, file))